# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                                           # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                           # [1], adaptive mixing
                                           # [1, K, L, M] # advanced adaptive mixing.
                                           # First, it does linear mixing for M steps. Mixing coeff is K the first 5 SCF steps.
                                           # Then it incrementally goes to L and becomes L at M-5 SCF step. From M-5 to M it's equal to L. After M'th SCF step, adaptive mixing begins.
                                           # 
                                           # [2], adaptive mixing, then pulay
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   401.84   1.038847e-01 -4.755824e+02 -4.754785e+02 ||    361.02   9.333198e-02 -1.091261e+02 -1.090327e+02 || 
     2   382.41   9.885982e-02 -4.755773e+02 -4.754784e+02 ||    383.49   9.914011e-02 -1.091319e+02 -1.090328e+02 || 
     3   344.90   8.916405e-02 -4.755675e+02 -4.754783e+02 ||    463.45   1.198119e-01 -1.091528e+02 -1.090330e+02 || 
     4   295.95   7.650900e-02 -4.755547e+02 -4.754782e+02 ||    588.52   1.521439e-01 -1.091855e+02 -1.090333e+02 || 
     5   244.18   6.312623e-02 -4.755411e+02 -4.754780e+02 ||    739.25   1.911116e-01 -1.092249e+02 -1.090337e+02 || 
     6   198.20   5.123786e-02 -4.755291e+02 -4.754779e+02 ||    891.53   2.304774e-01 -1.092647e+02 -1.090342e+02 || 
     7   164.43   4.250750e-02 -4.755204e+02 -4.754779e+02 ||   1019.84   2.636486e-01 -1.092982e+02 -1.090346e+02 || 
     8   145.63   3.764739e-02 -4.755155e+02 -4.754779e+02 ||   1101.42   2.847388e-01 -1.093196e+02 -1.090349e+

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created
## *MD_BOMD_BASIC.0.h5* and *MD_BOMD_BASIC.1.h5* are HDF5 files that contain all the data of the molecular dynamics trajectory

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [ ]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   604.90   1.563775e-01 -4.743654e+02 -4.742090e+02 ||    276.65   7.151890e-02 -1.083903e+02 -1.083188e+02 || 
     2   907.67   2.346497e-01 -4.744437e+02 -4.742090e+02 ||    329.89   8.528351e-02 -1.084040e+02 -1.083188e+02 || 
     3  1303.72   3.370390e-01 -4.745461e+02 -4.742090e+02 ||    559.10   1.445396e-01 -1.084633e+02 -1.083188e+02 || 
     4  1784.46   4.613188e-01 -4.746704e+02 -4.742090e+02 ||    950.73   2.457824e-01 -1.085645e+02 -1.083188e+02 || 
     5  2335.05   6.036564e-01 -4.748127e+02 -4.742090e+02 ||   1477.57   3.819819e-01 -1.087007e+02 -1.083188e+02 || 
     6  2932.48   7.581051e-01 -4.749672e+02 -4.742090e+02 ||   2097.26   5.421835e-01 -1.088609e+02 -1.083188e+02 || 
     7  3544.22   9.162514e-01 -4.751253e+02 -4.742090e+02 ||   2750.67   7.111023e-01 -1.090299e+02 -1.083188e+02 || 
     8  4127.86   1.067135e+00 -4.752762e+02 -4.742090e+02 ||   3361.06   8.689015e-01 -1.091877e+02 -1.083188e+

## Scale velocities to control temperature

In [ ]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp = 400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   1.034080e-01 -4.743639e+02 -4.742605e+02 ||    400.00   1.034080e-01 -1.084432e+02 -1.083398e+02 || 
     2   400.00   1.034080e-01 -4.744326e+02 -4.743292e+02 ||    400.00   1.034080e-01 -1.085017e+02 -1.083983e+02 || 
     3   400.00   1.034080e-01 -4.745108e+02 -4.744074e+02 ||    400.00   1.034080e-01 -1.085826e+02 -1.084792e+02 || 
     4   400.00   1.034080e-01 -4.745932e+02 -4.744897e+02 ||    400.00   1.034080e-01 -1.086731e+02 -1.085697e+02 || 
     5   400.00   1.034080e-01 -4.746764e+02 -4.745729e+02 ||    400.00   1.034080e-01 -1.087656e+02 -1.086622e+02 || 
     6   400.00   1.034080e-01 -4.747584e+02 -4.746550e+02 ||    400.00   1.034080e-01 -1.088560e+02 -1.087526e+02 || 
     7   400.00   1.034080e-01 -4.748382e+02 -4.747348e+02 ||    400.00   1.034080e-01 -1.089420e+02 -1.088386e+02 || 
     8   400.00   1.034080e-01 -4.749151e+02 -4.748117e+02 ||    400.00   1.034080e-01 -1.090225e+02 -1.089191e+

## Langevin Thermostat

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   385.43   1.992844e-01 -4.755786e+02 -4.753793e+02 ||    479.18   2.477562e-01 -1.091517e+02 -1.089039e+02 || 
     2   349.75   1.808329e-01 -4.755644e+02 -4.753835e+02 ||    532.85   2.755026e-01 -1.091812e+02 -1.089057e+02 || 
     3   311.52   1.610685e-01 -4.755408e+02 -4.753797e+02 ||    588.20   3.041255e-01 -1.092163e+02 -1.089122e+02 || 
     4   247.08   1.277496e-01 -4.755138e+02 -4.753860e+02 ||    623.73   3.224944e-01 -1.092504e+02 -1.089279e+02 || 
     5   209.01   1.080687e-01 -4.754863e+02 -4.753782e+02 ||    687.66   3.555471e-01 -1.092777e+02 -1.089222e+02 || 
     6   156.77   8.105450e-02 -4.754632e+02 -4.753822e+02 ||    704.93   3.644745e-01 -1.092979e+02 -1.089334e+02 || 
     7   153.31   7.926794e-02 -4.754496e+02 -4.753703e+02 ||    670.05   3.464444e-01 -1.093024e+02 -1.089560e+02 || 
     8   147.12   7.606685e-02 -4.754447e+02 -4.753686e+02 ||    700.70   3.622881e-01 -1.092884e+02 -1.089261e+